### Loaded: Github Functions
for getting average time to merge / respond to PRs, number of new tickets

In [0]:
import github
import datetime
import re

In [0]:
def filter_events(events: list, allowedEvents: list):
    result = []
    for event in events:
        if event.event in allowedEvents:
            result.append({
                "event":event.event
                ,"actor.login":event.actor.login
                ,"created_at":event.created_at
            })

    return result

In [0]:
def get_issues(issues_pag: github.PaginatedList.PaginatedList,
                  start_date: datetime.datetime,
                  end_date: datetime.datetime,
                  team_ids: list,
                  event_types: list,
                  final_event: str
                 ):
    '''
    Function for collecting issues from a paginated Github list and re-packaging these data
    according to desired KPIs:
    time it took a specific group of users to respond, time created.
    Feeds into 'parse_issues' function
    '''
    
    target_issues = []

    #Now pulling issues from Github. For each issue, creating a new structure 'pr_event' with meta-data if the event is 'merged', 'commented', or 'labeled' and
    #the event involved a member of the Wrangler team (not sure what actor.login is yet)
    for issue in issues_pag:
        issue_number = issue.number
        issue_created_at = issue.created_at
        
        if issue.created_at < start_date:
            break
        
        target_events = filter_events(events = issue.get_timeline(), allowedEvents = event_types)

        try:
            first_activity_from_team = next(event['created_at'] for event in target_events if event['actor.login'] in team_ids)
        except:
            first_activity_from_team = None

        if first_activity_from_team: #at least one event from wrangler
            new_issue = {
                "issue_number":issue_number
                ,"created_at":issue_created_at
                ,"time_first_team_activity":first_activity_from_team
                ,"time_to_first_response":first_activity_from_team - issue_created_at
            }
            
            try:
                first_merge_activity = next(event['created_at'] for event in target_events if event['event'] == final_event)
                new_issue[f"time_to_{final_event}"] = first_merge_activity - issue_created_at
            except:
                pass
            
            target_issues.append(new_issue)

    return target_issues


In [0]:
def parse_issues(issues_list: list, date_pairs: list, final_event: str, type: str):
    '''
    Function for taking a list of Github issues and aggregating some metrics by day:
    number of issues per day, time to respond to issues.
    Depends on custom list output from 'get_issues' function
    '''
    
    result = []
    final_event_key = f"time_to_{final_event}"
    
    for start_date, end_date in date_pairs:
        num_events = 0
        num_final_events = 0
        response_time = datetime.timedelta(0)
        final_event_time = datetime.timedelta(0)
        for issue in issues_list:
            if start_date <= issue['created_at'] <= end_date:
                num_events += 1
                response_time = response_time + issue['time_to_first_response']
                if final_event_key in issue.keys():
                    num_final_events += 1
                    final_event_time = final_event_time + issue[final_event_key]

        if num_events != 0:
            new_row = {
                "date":start_date.strftime("%Y-%m-%d")
                ,f"github_{type}_count":num_events
                ,f"github_{type}_days_to_first_response":response_time.total_seconds() / (60*60*24)
                #,"github_{type}_days_to_first_response_avg":(response_time/num_events).total_seconds() / (60*60*24)
            }
            
            if num_final_events != 0:
                new_row.update({
                    f"github_{type}_days_to_{final_event}":final_event_time.total_seconds() / (60*60*24)
                    #,f"github_{type}_{final_event}_days_avg":(final_event_time/num_final_events).total_seconds() / (60*60*24)
                })
                
            result.append(new_row)

    return result

In [0]:
def get_pr_merge_response_time(gh_client, team_wrangler, start_date, end_date, date_pairs):
    
    processing_repo = gh_client.get_repo("wrangler/processing-platform")
    processing_repo_issues = processing_repo.get_issues(state='closed', sort='created', direction='desc')
    
    final_event = "merged"
    
    processing_results = get_issues(issues_pag = processing_repo_issues
                                    ,start_date = start_date
                                    ,end_date = end_date
                                    ,team_ids = team_wrangler
                                    ,event_types = [final_event,"commented","labeled"]
                                    ,final_event = final_event
                                   )
    
    processing_results_parsed = parse_issues(issues_list = processing_results, date_pairs = date_pairs, final_event = final_event, type = "prs")
    
    return processing_results_parsed

In [0]:
def get_new_tickets_response_time(gh_client, team_wrangler, start_date, end_date, date_pairs):
    
    processing_repo = gh_client.get_repo("wrangler/issues")
    processing_repo_issues = processing_repo.get_issues(state='closed', sort='created', direction='desc')
    
    final_event = "closed"
    
    processing_results = get_issues(issues_pag = processing_repo_issues
                                    ,start_date = start_date
                                    ,end_date = end_date
                                    ,team_ids = team_wrangler
                                    ,event_types = [final_event,"commented","labeled"]
                                    ,final_event = final_event
                                   )
    
    processing_results_parsed = parse_issues(issues_list = processing_results, date_pairs = date_pairs, final_event = final_event, type = "issues")
    
    return processing_results_parsed